In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine, inspect, text

# Configs
DB_USER = 'root'
DB_PASSWORD = 'password'
DB_HOST = 'mysql-container'
DB_PORT = '3306'
DB_NAME = 'mydatabase'
TABLE_NAME = 'my_table'
SOURCE_FILE = 'data/source_file.csv'  # Can be .csv or .xlsx

# Create engine
engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

# Read the file
def read_file(file_path):
    ext = os.path.splitext(file_path)[1]
    if ext == '.csv':
        return pd.read_csv(file_path)
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path)
    else:
        raise Exception("Unsupported file format")

# Update schema if new columns are found
def update_table_schema(df):
    inspector = inspect(engine)
    columns = []
    if inspector.has_table(TABLE_NAME):
        columns = [col['name'] for col in inspector.get_columns(TABLE_NAME)]
    
    with engine.connect() as conn:
        for col in df.columns:
            if col not in columns:
                alter_sql = f"ALTER TABLE {TABLE_NAME} ADD COLUMN `{col}` TEXT"
                conn.execute(text(alter_sql))
                print(f"Added new column: {col}")

# Insert data
def insert_data(df):
    df.to_sql(TABLE_NAME, con=engine, if_exists='append', index=False)

def main():
    df = read_file(SOURCE_FILE)
    update_table_schema(df)
    insert_data(df)
    print("Data inserted successfully.")

if __name__ == "__main__":
    main()


Data inserted successfully.
